In [1]:
import os
from datasets import load_dataset
from segment.create_dataset import CreateSegmentationDataset
from segment.utils import load_resize_image
import random
from datasets import Dataset
from huggingface_hub import create_repo

/Users/jordandavis/GitHub/SEGMENT/GroundingDINO/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [ ]:
# import yaml
# # loading yaml config file
# @lru_cache(maxsize=None)
# def load_yaml(path):
#     with open(path, "r") as file:
#         data = yaml.load(file, Loader=yaml.FullLoader)
#     return data

# # Get the labels dictionary from the config file
# @lru_cache(maxsize=None)
# def get_labels_dict(config_path):
#     data = load_yaml(config_path)
#     labels_dict = data.get("names")
#     labels_dict = {v: k for k, v in labels_dict.items()}
#     return labels_dict

# class SegmentTrain:
#     def __init__(self):
#         pass

#     def create_yolo_config(self, save_path: str):
#         """Create and save the YOLO config YAML file."""
#         config = {"names": {i: label for label, i in self.labels_dict.items()}}
#         with open(save_path, "w") as file:
#             yaml.dump(config, file)
    # def _get_labels_dict(self) -> Dict[str, int]:
    #     """Get labels dictionary from the text prompt."""
    #     labels = self.text_prompt.split(".")
    #     return {label: i for i, label in enumerate(labels)}


In [ ]:
from segment.filters import Filter, score_filter, remove_empty_items
from segment.visualizer import visualizer
from typing import Union, List

    
class Segment(CreateSegmentationDataset):
    def __init__(self, ds: Dataset, text_prompt: Union[List, str], image_col='image', md_col='metadata', **kwargs):
        super().__init__(ds, text_prompt, **kwargs)
        self.ds = ds
        self.text_prompt = text_prompt
        self.image_col = image_col
        self.md_col = md_col

    def display(self, row_num=None):
        if row_num is None:
            row_num = random.randint(0, len(self.ds) - 1)

        row = self.ds[row_num]
        image = row[self.image_col]
        metadata = row[self.md_col]

        if not bool(metadata):
            raise ValueError("No metadata found for this image")

        visualizer(image, metadata)

    def filter_scores(self, score_cutoff: float):
        """
        Filter the processed dataset based on segmentation scores.

        Args:
            score_cutoff (float): The minimum score to keep a segmentation result.

        Raises:
            AssertionError: If there is no processed dataset to filter.
        """
        assert self.ds is not None, "There is no processed dataset to filter"

        self.ds = Filter.filter_list_in_column(
            self.ds,
            self.md_col,
            lambda x: score_filter(x, score_cutoff)
        )


    def remove_empty_rows(self):
        """
        Remove rows with empty metadata.
        """
        
        self.ds = Filter.filter_rows_of_dataset(
            self.ds,
            lambda x: remove_empty_items(x, self.md_col)
        )

    def push_to_hub(self, repo_id, token, commit_message="md", private=True):
        create_repo(
            repo_id=repo_id,
            repo_type="dataset",
            exist_ok=True,
            private=private,
            token=token,
        )

        self.ds.push_to_hub(
            repo_id, commit_message=commit_message, token=token
        )

        print(f"Pushed Dataset to Hub: {repo_id}")

In [2]:
image_dir = "datasets/fashion_people_detection/images/val"

ds = load_dataset("imagefolder", data_dir=image_dir, split="train")
text_prompt = ["face", "glasses", "clothes"]
ds = ds.shuffle().take(2)

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:

seg = Segment(ds, text_prompt, sub_batch_size=1)

In [ ]:
seg.process()

In [ ]:
seg.display()